<a href="https://colab.research.google.com/github/swapnilbotu/FLCMLBootcamp25Swapnil/blob/main/3_Basics_PythonProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digital Grocery Basket


## Problem statement

#### Create a simple digital grocery basket using which a grocer can process check-out at a grocery store and generate a receipt:
- Add items to the basket.
- Remove items from the basket.
- Update the quantity of items in the basket.
- Display the current basket contents.
- Save and load the basket data using JSON for persistence.
- Once all the items are added, display a receipt of the purchase.
- Track transactions and manage inventory

## Initialize the inventory of the grocery mart

In [113]:
import json
import os
import pandas as pd
import requests
from IPython.display import display # Import display

In [114]:
# Fetch the JSON data from the URL
url = 'https://raw.githubusercontent.com/shashankbl/mlbootcamp2025flc/main/extras/grocery_inventory.json'  # Use 'raw.githubusercontent.com'
try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Error fetching JSON from URL: {e}")
    data = {}  # Initialize as empty dictionary in case of error

# Convert the JSON data to a Pandas DataFrame
try:
    df = pd.DataFrame(data)
    display(df) # Display the DataFrame using display() for pretty print
except (ValueError, KeyError) as e:
    print(f"Error converting JSON to DataFrame: {e}")
    if data:
        print("JSON data loaded but could not be formatted as a table")

,item,category,quantity,unit_cost,max_quantity
0,tomato,vegetable,100,0.10,10
1,apple,fruit,54,0.25,5
2,oranges,fruit,40,0.05,10
3,strawberry,fruit,430,0.08,15
4,onion,vegetable,254,0.20,8
5,carrot,vegetable,204,0.05,15
6,pepper,vegetable,540,0.10,15
7,eggplant,vegetable,50,0.30,5
8,broccoli,vegetable,67,2.50,2
9,beans,vegetable,35,2.00,3


In [115]:
# Understand the list of items under each category
groupes = df.groupby('category')

for category, group in groupes:
    print(f"\nCategory: {category}")
    print(list(group['item']))

    # TODO: Write code below to calculate and print the number of items under each category
    print("Number of items in this category:", len(group))


Category: beverages
['soda', 'sparkling_water']
Number of items in this category: 2

Category: dairy
['milk', 'yogurt', 'cheese']
Number of items in this category: 3

Category: dessert
['icecream', 'mochi']
Number of items in this category: 2

Category: fruit
['apple', 'oranges', 'strawberry']
Number of items in this category: 3

Category: vegetable
['tomato', 'onion', 'carrot', 'pepper', 'eggplant', 'broccoli', 'beans']
Number of items in this category: 7


In [116]:
df["item"].values

array(['tomato', 'apple', 'oranges', 'strawberry', 'onion', 'carrot',
       'pepper', 'eggplant', 'broccoli', 'beans', 'milk', 'yogurt',
       'cheese', 'icecream', 'mochi', 'soda', 'sparkling_water'],
      dtype=object)

## Digital Grocery Basket

In [117]:
# Python class called GroceryBasket
class GroceryBasket:
    def __init__(self, filename="basket.csv"):
        self.filename = filename
        self.basket = self.load_basket()

    def load_basket(self):
        """Load basket from basket.csv"""
        try:
            basket_df = pd.read_csv(self.filename)

            # Convert DataFrame to dictionary: key=item, value=quantity
            basket = dict(zip(basket_df['item'], basket_df['quantity']))
            return basket
        except Exception as e:
            print(f"Error loading basket from file: {e}")
            return {}

    def save_basket(self):
        """Save basket to basket.csv"""
        basket_df = pd.DataFrame(list(self.basket.items()), columns=['item', 'quantity'])
        basket_df.to_csv(self.filename, index=False)

    def add_item(self, item, quantity):
        """Add an item or update quantity"""
        if item in self.basket:
            self.basket[item] += quantity
        else:
            self.basket[item] = quantity
        self.save_basket()

    def remove_item(self, item):
        """Remove an item from the basket"""
        if item in self.basket:
            del self.basket[item]
            self.save_basket()
        else:
            print("Item not in basket.")

    def update_quantity(self, item, quantity):
        """Update the quantity of an existing item."""
        if item in self.basket:
            self.basket[item] = quantity
            self.save_basket()
        else:
            print("Item not in basket.")

    def view_basket(self):
        """Display the current basket contents."""
        if self.basket:
            print("Current basket contents:")
            for item, quantity in self.basket.items():
                print(f"{item}: {quantity}")
        else:
            print("Basket is empty.")

    def clear_basket(self):
        """Clear all items in the basket."""
        self.basket = {}
        self.save_basket()
        print("Basket cleared.")

In [125]:
basket = GroceryBasket()

while True:
    print("\n--- Choose an option ---")
    print("1=Add item, 2=Remove item, 3=Update quantity, 4=View basket, 5=Clear basket, 6=Checkout")

    # NOTE: Users can only shop items already present in the initial inventory
    # Take user input and check if it is a valid input
        # make sure for remaining inventory, max quantity per transaction,...
    # If user input is valid, conduct actions with the digital grocery basket
    # If user input is invalid, provide a clear prompt mentioning the error
    # Exit the while loop once user opts to checkout

    # TODO:
    userChoice = input("Enter your choice number: ").strip()

    if userChoice == '1':
        print("Items and quantities available to purchase:\n\n", df[["item","max_quantity"]])
        item = input("Enter item name to add: ").strip()
        # make sure item is in the inventory
        if item not in df['item'].values:
            print("Item not found in store.")
            continue
        try:
            quantity = int(input("Enter quantity: "))
        except ValueError:
            print("Invalid quantity.")
            continue

        # check if quantity exceeds the maximum allowed per transaction for the item
        inv_row = df[df['item'] == item]
        if not inv_row.empty:
            max_allowed = inv_row.iloc[0]['max_quantity']
            if quantity > max_allowed:
                print(f"Requested quantity exceeds maximum allowed per transaction ({max_allowed}).")
                continue
        basket.add_item(item, quantity)
        print(f"Added {quantity} {item}'s to basket.")

    elif userChoice == '2':
        item = input("Enter item name to remove: ").strip()
        basket.remove_item(item)

        if item in df['item'].values:
            print(f"Removed {item} from basket")
        else:
            print(f"{item} not found in current basket")

    elif userChoice == '3':
        print(basket.view_basket())
        item = input("Enter item name to update: ").strip()
        if item not in basket.basket:
            print("Item not in basket.")
            continue
        try:
            quantity = int(input("enter new quantity: "))
        except ValueError:
            print("Invalid quantity.")
            continue

        # check if new quantity exceeds the maximum allowed per transaction for the item
        inv_row = df[df['item'] == item]
        if not inv_row.empty:
            max_allowed = inv_row.iloc[0]['max_quantity']
            if quantity > max_allowed:
                print(f"Requested quantity exceeds maximum allowed per transaction ({max_allowed}).")
                continue
        basket.update_quantity(item, quantity)
        print(f"Updated {item} quantity to {quantity}.")

    elif userChoice == '4':
        basket.view_basket()

    elif userChoice == '5':
        basket.clear_basket()

    elif userChoice == '6':
        print("Proceeding to checkout...")
        break

    else:
        print("Invalid option. Please choose a valid option.")


--- Choose an option ---
1=Add item, 2=Remove item, 3=Update quantity, 4=View basket, 5=Clear basket, 6=Checkout
Enter your choice number: 1
Items and quantities available to purchase:

                item  max_quantity
0            tomato            10
1             apple             5
2           oranges            10
3        strawberry            15
4             onion             8
5            carrot            15
6            pepper            15
7          eggplant             5
8          broccoli             2
9             beans             3
10             milk             3
11           yogurt             5
12           cheese             3
13         icecream            10
14            mochi             5
15             soda            15
16  sparkling_water            12
Enter item name to add: tomato
Enter quantity: 15
Requested quantity exceeds maximum allowed per transaction (10).

--- Choose an option ---
1=Add item, 2=Remove item, 3=Update quantity, 4=View basket

In [126]:
basket.view_basket()

Current basket contents:
milk: 3
cheese: 3


In [127]:
# todo: generate the final receipt for the transaction and also log the transaction into a transaction ledger

# creating the receipt
print("Receipt:")
grand_total = 0.0
receipt_items = []  # to hold details of each item purchased

for item, quantity in basket.basket.items():
    # get unit price from inventory (assumes a 'price' column exists)
    row = df[df['item'] == item]
    if not row.empty:
        unit_price = row.iloc[0]['unit_cost']
        total_price = unit_price * quantity
        grand_total += total_price
        print(f"{item}: {quantity} x ${unit_price:.2f} = ${total_price:.2f}")
        receipt_items.append({
            'item': item,
            'quantity': quantity,
            'unit_price': unit_price,
            'total': total_price
        })
print(f"Grand Total: ${round(grand_total,2)}")


# log the transaction into a transaction ledger
from datetime import datetime

ledger_file = "transaction_ledger.csv"

# get the current timestamp for the transaction
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# create a list to hold each ledger row (one row per purchased item)
ledger_rows = []
for item in receipt_items:
    ledger_rows.append({
        "time": timestamp,
        "item": item["item"],
        "quantity": item["quantity"],
        "unit_price": item["unit_price"],
        "total_price": item["total"]
    })

# create a dataframe from the list of rows
transaction_df = pd.DataFrame(ledger_rows)

# define the ledger csv file name.
# for persistent storage in google colab, consider mounting google drive and adjusting the path.
ledger_file = "transaction_ledger.csv"

# check if the ledger file exists and load existing data if it does.
if os.path.exists(ledger_file):
    ledger_df = pd.read_csv(ledger_file)
    # append the new transaction rows
    ledger_df = pd.concat([ledger_df, transaction_df], ignore_index=True)
else:
    # create a new dataframe if the ledger doesn't exist
    ledger_df = transaction_df

# save the updated ledger to csv
ledger_df.to_csv(ledger_file, index=False)
print("\nTransaction has been logged to:", ledger_file)

Receipt:
milk: 3 x $3.50 = $10.50
cheese: 3 x $3.00 = $9.00
Grand Total: $19.5

Transaction has been logged to: transaction_ledger.csv


In [128]:
# TODO: Generate the remaining inventory after the transaction is completed
for item, quantity in basket.basket.items():
    if item in df['item'].values:
        df.loc[df['item'] == item, 'quantity'] -= quantity

# ensure no negative stock values
df['quantity'] = df['quantity'].clip(lower=0)

# print the updated inventory
print("Updated Inventory:")
print(df.to_string(index=False))  # prints the full dataframe without truncation

# clearing the basket for the next customer
basket.clear_basket()

Updated Inventory:
           item  category  quantity  unit_cost  max_quantity
         tomato vegetable        97       0.10            10
          apple     fruit        54       0.25             5
        oranges     fruit        40       0.05            10
     strawberry     fruit       430       0.08            15
          onion vegetable       254       0.20             8
         carrot vegetable       204       0.05            15
         pepper vegetable       540       0.10            15
       eggplant vegetable        50       0.30             5
       broccoli vegetable        67       2.50             2
          beans vegetable        35       2.00             3
           milk     dairy        51       3.50             3
         yogurt     dairy        60       5.00             5
         cheese     dairy        82       3.00             3
       icecream   dessert       150       2.00            10
          mochi   dessert        70       1.50             5
     

In [129]:
# TODO: Save all the data tables as csv
df.to_csv("updated_inventory.csv", index=False)

print("\nAll data tables have been saved as CSV files.")


All data tables have been saved as CSV files.
